# BERT를 이용하여 임베딩 생성하기
---


In [ ]:
!pip install transformers

In [ ]:
# 필요 라이브러리 불러오기
from transformers import AutoTokenizer, AutoModel
import torch 

In [ ]:
# 사전학습된 모델 불러오기
model = AutoModel.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# bert-base-multilingual-cased를 사전 학습시키는데 사용 된 토크나이저 불러오기
# 모델을 불러올 때 bert-base-multilingual-cased를 불러왔기 때문에 tokenizer도 동일한 것으로 불러와야됨.
# 아니면 모델이 정상적으로 작동하지 않음
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

# 입력 전처리 하기
1. 문장을 토큰화 한다.
2. 토큰들의 시작 부분에 `[CLS]` 토큰을 추가하고 토큰 마지막 부분에 `[SEP]`토큰을 추가한다.
3. attention mask를 만든다.
4. 토큰을 토큰ID(숫자)로 변환

In [ ]:
# 입력 문장 정의
sentence = '제주도는 정말 아름답다'

In [ ]:
# 문장을 토큰화 한다.
tokens = tokenizer.tokenize(sentence)
# 토큰 출력
print(tokens)

['제', '##주', '##도는', '정', '##말', '아', '##름', '##답', '##다']


In [ ]:
# 시작 부분에 [CLS]토큰 마지막 부분에 [SEP]토큰 추가
tokens = ['[CLS]'] + tokens + ['[SEP]']
#토큰 출력
print(tokens)

['[CLS]', '제', '##주', '##도는', '정', '##말', '아', '##름', '##답', '##다', '[SEP]']


In [ ]:
# attention mask생성
# 토큰에 [PAD] 토큰이 있다면 해당부분은 0으로 채워주고 그렇지 않다면 1로 채워주면됨.
# 입력 문장에 [PAD] 토큰이 없기 때문에 모두 1로 채워줌
attention_mask = [1,1,1,1,1,1,1,1,1,1,1]

In [ ]:
# 모든 토큰을 토큰 ID로 변환
token_ids = tokenizer.convert_tokens_to_ids(tokens)
# token_ids출력
print(token_ids)

[101, 9672, 16323, 60884, 9670, 89523, 9519, 49543, 118775, 11903, 102]


In [ ]:
# token_ids와 attention_mask를 텐서로 변환
# bert model에 넣어주기 위해서는 tensor로 변환해 주어야 함.
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

# 임베딩 추출
toekn_ids 및 attention_mask를 모델에 입력하고 임베딩을 얻을 수 있다.   
모델은 은닉상태 표현과 [CLS] 토큰의 표현으로 구성된다.

In [ ]:
output = model(token_ids,attention_mask)

In [ ]:
hidden_rep = output[0]
cls_head = output[1]

In [ ]:
# hidden_rep의 크기는 [1, 11, 768] 인데 1은 배치크기, 11은 입력 시퀀스의 길이, 768은 임베딩 크기를 의미한다.
print(hidden_rep.shape)

torch.Size([1, 11, 768])


- hidden_rep[0][0] : 첫 번째 토큰인 `[CLS]`의 표현 벡터
- hidden_rep[0][1] : 두 번째 토큰인 `제`의 표현 벡터

In [ ]:
# cls_head의 출력값 중 1은 배치사이즈를 의미하고 768은 임베딩 차원이다.
# cls_head가 입력 문장의 표현 벡터이다.
print(cls_head.shape)

torch.Size([1, 768])
